In [ ]:
import json

import openai
import numpy as np
import pandas as pd
from pydantic import BaseModel

In [ ]:
with open("/Users/vanzettom/Documents/mattia/personale/hackapizza/openai_api_mattiavanzetto.txt", encoding='utf-8') as file:
    api_key = file.read()

In [ ]:
with open("/Users/vanzettom/Documents/mattia/personale/hackapizza/data_preprocessed/plate_info_dictionary.json") as f:
    plate_info_dictionary = json.load(f)

In [ ]:
plate_info_dictionary['Quadrifonia Cosmica: Sinfonia di Sapori e Dimensioni']

In [ ]:
questions = pd.read_csv('/Users/vanzettom/Documents/mattia/personale/hackapizza/Hackapizza Dataset/domande.csv')
questions.shape

In [ ]:
with open('/Users/vanzettom/Documents/mattia/personale/hackapizza/Hackapizza Dataset/Misc/dish_mapping.json') as f:
    dish_mapping = json.load(f)

In [ ]:
[p for p in list(plate_info_dictionary.keys()) if p not in [d for d in dish_mapping]]

In [ ]:
with open("/Users/vanzettom/Documents/mattia/personale/hackapizza/data_preprocessed/license_list.json") as f:
    license_list = json.load(f) 

In [ ]:
license_list_short = """
- "Licenza Psionica (P)"
- "Licenza Temporale (t)"
- "Licenza Gravitazionale (G)"
- "Licenza Antimateria (e+)"
- "Licenza Magnetica (Mx)"
- "Licenza Quantistica (Q)"
- "Licenza Luce (c)"
- "Livello di Sviluppo Tecnologico (LTK)"
"""

In [ ]:
print(license_list_short)

In [ ]:
import itertools

total_ingredient_set = set(itertools.chain(*[d['ingredienti'] for d in plate_info_dictionary.values()]))
len(total_ingredient_set)

In [ ]:
total_ingredient_set

### Riformulazione domande

In [ ]:
%%time

client = openai.OpenAI(
  api_key=api_key
)

model = "gpt-4o"

new_questions = dict()

for i, query in enumerate(questions['domanda'], start=1):
    
    print(f"{i} ---- original question:\n\n{query}")
    
    output = client.beta.chat.completions.parse(
    model=model,
    store=True,
    messages=[
        {
            "role": "system",
            "content": """
Sei un esperto culinario che aiuta a riformulare domande poste da utenti in modo che siano più chiare per poi essere gestite da un sistema AI.
In particolare ogni domanda chiede quali piatti di cucina, tratti da dei menù diversi, soddisfano certe condizioni.
I piatti non sono realistici ma sono inventati in un contento fantascientifico spaziale, quindi sia gli ingredienti che le tecniche utilizzate per preparare i piatti sono inventati.
Il tuo compito a ogni domanda devi estrarre i requisiti che i piatti devono avere.
"""
        },
        {
            "role": "user",
            "content": f"""
            
### INDICAZIONI:           
La domanda che ti mostrerò chiede quali piatti di un menu rispettano certe condizioni.
Voglio che la riformuli riferendola al singolare, quindi riferendola specificatamente ad un piatto, chiedendo se un piatto specifico rispetta le condizioni racchiuse nella domanda originale.
Voglio che scomponi la domanda originale in un elenco puntato di requisiti che il piatto deve soddisfare.
Voglio che elimini aggettivi superflui e che non rappresentano requisiti del piatto.
La domanda potrebbe contenere riferimenti a licenze che lo chef deve possedere riportando le licenze in forma abbreviata. Voglio che nel riformulare la domanda sostituisci le abbreviazioni con le licenze riportandole per esteso.
Di seguito ti riporto la lista di licenze completa che potresti trovare nella domanda (ad ogni licenza posseduta dallo chef è associato un livello). Inoltre ti riporto anche una lista non esaustiva contenente esempi di ingredienti che possono contenere i piatti. Infine ti mostrerò anche alcuni esempi pratici.

### ALTRE INFO UTILI:
- Sirius Cosmo è una persona che ha scritto un famoso manuale di cucina, che prende il nome dall’autore.
- I piatti sono inventati ed anche le tecniche e ingredienti che li compongono. Ci troviamo in un'ambientazione fantascientifica.

### LISTA LICENZE POSSIBILI:
{license_list_short}

### ESEMPI DI INGREDIENTI:
{', '.join(total_ingredient_set)}

### DOMANDA ESEMPIO 1:
"Quali piatti sono accompagnati dai misteriosi Frutti del Diavolo, che donano poteri speciali a chi li consuma?"

### SOLUZIONE ESEMPIO 1:
"Questo piatto soddisfa i seguenti requisti?:
-contiene tra gli ingredienti i 'frutti del diavolo'".

### DOMANDA ESEMPIO 2:
"Quali piatti usano la Sferificazione Filamentare a Molecole Vibrazionali, ma evitano la Decostruzione Magnetica Risonante?"

### SOLUZIONE ESEMPIO 2:
"Questo piatto soddisfa i seguenti requisti?:
-nella sua preparazione è necessaria la tecnica 'Sferificazione Filamentare a Molecole Vibrazionali'.
-nella sua preparazione è necessaria la tecnica 'Decostruzione Magnetica Risonante'".

### DOMANDA ESEMPIO 3:
"Quali piatti sono stati creati utilizzando almeno una tecnica di surgelamento secondo il di Sirius Cosmo e sono stati serviti in un ristorante situato entro un raggio di 317 anni luce dal pianeta Krypton, Krypton incluso?"

### SOLUZIONE ESEMPIO 3:
"Questo piatto soddisfa i seguenti requisti?:
-nella sua preparazione è necessaria almeno una tecnica di impasto del manuale di Sirius Cosmo.
-il ristorante in cui viene preparato dista massimo 317 anni luce da Krypton, oppure è situato su Krypton."

### DOMANDA ESEMPIO 4:
"Che piatti posso mangiare che contengano i Fusilli del Vento ma che abbia nella sua preparazione la Marinatura a Infusione Gravitazionale correttamente operata da un chef che ha le corrette licenze e certificazioni descritte dal Codice di Galattico?"

### SOLUZIONE ESEMPIO 4:
"Questo piatto soddisfa i seguenti requisti?:
-contiene tra gli ingredienti i 'fusilli del vento'
-nella sua preparazione è necessaria la tecnica 'Marinatura a Infusione Gravitazionale'.
-lo chef che prepara questo piatto possiede le corrette licenze per prepararlo, quindi è 'ok' con le licenze necessarie.
"

### DOMANDA ESEMPIO 5:
"Quali piatti creati con almeno una tecnica di taglio dal Manuale di Cucina di Sirius Cosmo e che necessitano della licenza t non base per la preparazione, escludendo quelli con Fusilli del Vento, sono serviti?"

### SOLUZIONE ESEMPIO 5:
"Questo piatto soddisfa i seguenti requisti?:
-nella sua preparazione è necessaria almeno una tecnica di taglio tratta dal manuale di Sirius Cosmo.
-nella sua preparazione è necessaria la "Licenza Temporale (t)" di livello superiore a 1.
-non contiene tra gli ingredienti i 'fusilli del vento'.
"

 ———-———-———-———-

Restituisci solo la seguente domanda riformulata, senza altri commenti da parte tua.

### DOMANDA:
{query}

### SOLUZIONE:

"""
        }
    ],
    )
    q = output.choices[0].message.content
    print(f"\n---- new question:\n\n{q}\n\n")
    new_questions[i] = q

In [ ]:
with open("/Users/vanzettom/Documents/mattia/personale/hackapizza/data_preprocessed/new_questions.json", "w", encoding="utf-8") as f:
    json.dump(new_questions, f, ensure_ascii=False, indent=4)

In [ ]:
len(new_questions)

In [ ]:
new_questions

### Validazione piatti

In [ ]:
example_dict = plate_info_dictionary['Quadrifonia Cosmica: Sinfonia di Sapori e Dimensioni']

In [ ]:
%%time

client = openai.OpenAI(
  api_key=api_key
)

model = "gpt-4o", #"o3-mini"

responses = dict()
functions = dict()

for i, query in new_questions.items():
    
    responses[i] = list()
    print(f"{i} - {query}")
    
    output = client.beta.chat.completions.parse(
    model=model,
    store=True,
    messages=[
        {
            "role": "system",
            "content": """
Sei un programmatore python esperto che sta aiutando a capire informaticamente se uno specifico piatto soddisfa o meno una richiesta di un cliente.
Le informazioni sul piatto ti saranno fornite sotto forma di dizionario python.
Ti trovi in una ambientazione fantascientifica nello spazio, in diversi sistemi solari.
Il tuo risultato deve essere la definizione di una funzione python che prende in input un dizionario contenente i dati di un piatto e restituisce soltanto True o False.
Non devono essere presenti altri commenti poiché il tuo risultato dovrà essere passato ad un exec per poi utilizzare la funzione.
"""
        },
        {
            "role": "user",
            "content": f"""
### INDICAZIONI GENERALI: 
- Qui sotto ti fornirò un esempio di dizionario contenente i dati di un piatto di un ristorante che dovrai valutare per capire se soddisfa o meno i requisiti di una specifica domanda di un utente.
- Voglio da te che produci una funzione python che prende in input soltanto il dizionario e restituisce True o False in base a se il piatto soddisfa o meno la domanda dell'utente.
- Le informazioni relativamente alla domanda (che ti fornirò qui sotto) devono essere completamente hardcodate all'interno della funzione, poiché la funzione deve prendere in input solo il dizionario.

### INDICAZIONI PER PRODURRE LA FUNZIONE PYTHON:
- Considera che alcune parole della domanda potrebbero contenere typo e che le rispettive chiavi o valori che ti servono nel dizionario potrebbero anch'esse contenere typo. Per questo devi usare la distanza di Levenshtein dove lo ritieni più opportuno nella tua funzione (impostando distanza massima di 3). 
- Usa sempre il .lower() quando cerchi di verificare alcune condizioni.
- Eventuali import di librerie inseriscili direttamente dentro la funzione.

### INFORMAZIONI SUI DATI CHE AVRAI A DISPOSIZIONE:
- La lista di ingredienti necessari per i piatti la trovi nella chiave 'ingredienti', il nome del pianeta in ["dati_ristorante"]["pianeta"] e così via...
- Le distanze tra pianeti sono espresse in anni luce.
- Ogni dizionario contiene le licenze possedute dallo chef del ristorante in cui viene cucinato il piatto.
- Ogni licenza è associata ad un livello. 
- Ogni piatto richiede delle tecniche particolari per essere preparato, la lista di tecniche necessarie la trovi nella chiave 'tecniche_utilizzate'.
- Ogni piatto richiede che lo chef possegga certe licenze, di un certo licello, per essere prepararato, la lista di licenze necessarie la trovi nella chiave 'licenze_necessarie_per_la_preparazione'. Nonostante questo uno chef può cucinare ipoteticamente un piatto anche se non possiede tutte le richieste necessarie (puoi capire se le possiede nella chiave 'licenze_ok').
- Nella galassia dove si trovano questi ristoranti è diffuso un manuale di cucina prodotto da Sirius Cosmo.
- Un piatto potrebbe richiedere l'utilizzo di una tecnica contenuta nel manuale di Sirius Cosmo, trovi le tecnice di Sirius Cosmo necessarie alla preparazione nella chiave 'tecniche_utilizzate_del_Sirius_Cosmo'.

Di seguito ti forniro esempio di dizionario, due esempi di domanda con relativi due esempi di soluzione.

### DIZIONARIO DI ESEMPIO:
{example_dict}

### DOMANDA ESEMPIO 1:
"Questo piatto soddisfa i seguenti requisiti?:
- nella sua preparazione è necessaria almeno una tecnica di surgelamento del manuale di Sirius Cosmo.
- il ristorante in cui viene preparato dista massimo 317 anni luce da Krypton, oppure è situato su Krypton."

### SOLUZIONE ESEMPIO 1:
"def verifica_piatto(d):
    
    conditions = list()
    
    # - nella sua preparazione è necessaria almeno una tecnica di surgelamento del manuale di Sirius Cosmo.
    conditions.append(
        len(d['tecniche_utilizzate_del_Sirius_Cosmo']['Tecniche di Preparazione']['tecniche di Surgelamento']) > 0
    )
    
    # - il ristorante in cui viene preparato dista massimo 317 anni luce da Krypton, oppure è situato su Krypton."
    conditions.append(
        (d['dati_ristorante']['distanze_da_altri_pianeti']['Krypton'] < 317)
        or
        (d['dati_ristorante']['pianeta'] == 'Krypton')
    )
    
    return all(conditions)"
    
### DOMANDA ESEMPIO 2:
"Questo piatto soddisfa i seguenti requisiti?:
- nella sua preparazione è necessaria almeno una tecnica di taglio tratta dal manuale di Sirius Cosmo.
- nella sua preparazione è necessaria la "Licenza Gravitazionale (G)" di livello 2 o superiore.
- non contiene tra gli ingredienti i 'gnocchi del crepuscolo'."


### SOLUZIONE ESEMPIO 2:
"def verifica_piatto(d):
    import Levenshtein
    
    conditions = list()
    
    # - nella sua preparazione è necessaria almeno una tecnica di taglio tratta dal manuale di Sirius Cosmo.
    conditions.append(
        len(d['tecniche_utilizzate_del_Sirius_Cosmo']['Tecniche Avanzate']['Tecniche di Taglio']) > 0
    )
    
    # - nella sua preparazione è necessaria la "Licenza Gravitazionale (G)" di livello 2 o superiore.
    licenze_filtrate = [l for l in d['licenze_necessarie_per_la_preparazione'] if 'Licenza Gravitazionale (G)' in l]
    licenze_filtrate = list(filter(lambda x: x['Licenza Gravitazionale (G)'] >= 2, licenze_filtrate))
    conditions.append(len(licenze_filtrate) > 0)
    
    # - non contiene tra gli ingredienti i 'gnocchi del crepuscolo'."
    lista_ingredienti = [ingrediente.lower().strip() for ingrediente in d['ingredienti']]
    contiene_ingrediente = any(Levenshtein.distance(ingrediente, "gnocchi del crepuscolo".lower().strip()) <= 3 for ingrediente in lista_ingredienti)
    conditions.append(not contiene_ingrediente)
    
    return all(conditions)"
    
----

Forniscimi in output solo la definzione di una funzione python che si chiama 'verifica_piatto', che prende in input solamente un dizionario, e che restituisce un booleano, in modo che possa inserirla in un exec().
Ricordati di usare i lower e la distanza di Levenshstein quando necessario.

Ragiona per step, leggi bene la **DOMANDA** e cerca di scomporla in requisiti che il piatto deve avere.

### DOMANDA:
{query}

### SOLUZIONE:

"""
        }
    ],
    )
    f = output.choices[0].message.content
    functions[i] = f
    f = f.replace('`', '').replace('python', '')
    try:
        exec(f)
        for plate, d in plate_info_dictionary.items():
            if verifica_piatto(d):
                responses[i].append(dish_mapping.get(plate))
    except:
        print("error in exec")
    print(f"Piatti idonei trovati: {len(responses[i])}")

In [ ]:
with open("/Users/vanzettom/Documents/mattia/personale/hackapizza/data_preprocessed/responses.json", "w", encoding="utf-8") as f:
    json.dump(responses, f, ensure_ascii=False, indent=4)

In [ ]:
responses_pd = pd.DataFrame({'row_id': responses.keys(), 'result': responses.values()})

In [ ]:
default_pred_list = [1,2,3]

responses_pd['result'] = responses_pd['result'].apply(lambda x: x if len(x) > 0 else default_pred_list)

In [ ]:
responses_pd['result'] = responses_pd['result'].apply(lambda x: ",".join([str(p) for p in x]).replace("None,", ""))

In [ ]:
pd.set_option('display.max_rows', 101)

In [ ]:
responses_pd

In [ ]:
responses_pd.to_csv(f"/Users/vanzettom/Documents/mattia/personale/hackapizza/submissions/submission_questions_redefined_v5.csv", index=False, sep=",")

### Piano B - RAG andando a cercare documenti specifici

In [ ]:
import openai
from pydantic import BaseModel

In [ ]:
with open("/Users/vanzettom/Documents/mattia/personale/hackapizza/data_preprocessed/plate_chunks_rag.json") as f:
    plate_chunks_rag = json.load(f)  

In [ ]:
with open("/Users/vanzettom/Documents/mattia/personale/hackapizza/data_preprocessed/manuale_sirius_cosmo.json") as f:
    manuale_sirius_cosmo = json.load(f)  

In [ ]:
with open("/Users/vanzettom/Documents/mattia/personale/hackapizza/data_preprocessed/distanze.json") as f:
    distanze = json.load(f)  

In [ ]:
with open("/Users/vanzettom/Documents/mattia/personale/hackapizza/data_preprocessed/sostanze_quantita_regolamentate.json") as f:
    sostanze_quantita_regolamentate = json.load(f)  

In [ ]:
with open("/Users/vanzettom/Documents/mattia/personale/hackapizza/data_preprocessed/licenze_necessarie_per_tecniche.json") as f:
    licenze_necessarie_per_tecniche = json.load(f)  

In [ ]:
with open("/Users/vanzettom/Documents/mattia/personale/hackapizza/data_preprocessed/sanzioni_pene.json") as f:
    sanzioni_pene = json.load(f)  

In [ ]:
with open("/Users/vanzettom/Documents/mattia/personale/hackapizza/data_preprocessed/blog_etere_del_gusto.json") as f:
    blog_etere_del_gusto = json.load(f)  

In [ ]:
with open("/Users/vanzettom/Documents/mattia/personale/hackapizza/data_preprocessed/blog_sapore_del_dune.json") as f:
    blog_sapore_del_dune = json.load(f)  

In [ ]:
import faiss
import tiktoken

client = openai.OpenAI(api_key=api_key)

def chunk_text(text, chunk_size=500, overlap=50):
    tokenizer = tiktoken.get_encoding("cl100k_base")
    tokens = tokenizer.encode(text)
    chunks = []
    for i in range(0, len(tokens), chunk_size - overlap):
        chunk_tokens = tokens[i:i + chunk_size]
        chunk_text = tokenizer.decode(chunk_tokens)
        chunks.append(chunk_text)
    return chunks

def create_vector_db(client, text, model="text-embedding-ada-002", chunk_size=500, overlap=50):
    texts = chunk_text(text, chunk_size, overlap)
    embeddings = np.array([get_embedding(client, t, model) for t in texts]).astype('float32')
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)
    return index, texts

def search_similar_chunks(client, query, index, texts, k=5, model="text-embedding-ada-002"):
    query_embedding = get_embedding(client, query, model).astype('float32').reshape(1, -1)
    distances, indices = index.search(query_embedding, k)
    return [texts[i] for i in indices[0]]


def get_embedding(client, text, model="text-embedding-ada-002"):
    response = client.embeddings.create(input=[text], model=model)
    return np.array(response.data[0].embedding)

def retrieve_top_k_from_dict(query, embeddings_dict, k=20):
    query_embedding = get_embedding(client, query)
    
    similarities = {
        key: np.dot(query_embedding, embedding) / (np.linalg.norm(query_embedding) * np.linalg.norm(embedding))
        for key, embedding in embeddings_dict.items()
    }
    
    top_k_keys = sorted(similarities, key=similarities.get, reverse=True)[:k]
    
    return top_k_keys

In [ ]:
# Creo gli embedding per le descrizioni dei piatti e li salvo in un dizionario (poi mi servirà la chiave)
descrizione_piatti_embeddings_dict = {key: get_embedding(client, value) for key, value in plate_chunks_rag.items()}

In [ ]:
# Creo un vector DB solo per il manuale di Sirius Cosmo + Lista di licenze necessarie per usare le tecniche, perché gli altri sono testi corti

index_SC, texts_SC = create_vector_db(client, manuale_sirius_cosmo, model="text-embedding-ada-002", chunk_size=500, overlap=50)

index_LNPT, texts_LNPT = create_vector_db(client, licenze_necessarie_per_tecniche, model="text-embedding-ada-002", chunk_size=500, overlap=50)

### Domande senza risposta

In [ ]:
responses_pd = pd.read_csv(f"/Users/vanzettom/Documents/mattia/personale/hackapizza/submissions/submission_questions_redefined_v5.csv",)

In [ ]:
question_remaining_ids = list(responses_pd.loc[responses_pd['result'] == '1,2,3', 'row_id'])
question_remaining_ids

### che documenti servono per rispondere alla domanda? + rag

In [ ]:
class DocNeeded(BaseModel):
    manuale_sirius_cosmo: bool
    distanze_pianeti: bool
    sostanze_quantita_regolamentate: bool
    licenze_necessarie_per_tecniche: bool
    sanzioni_pene: bool
    blog_etere_del_gusto: bool
    blog_sapore_del_dune: bool
    license_list: bool

In [ ]:
client = openai.OpenAI(
  api_key=api_key
)

remaining_answers_doc_needed = dict()

for id_query in question_remaining_ids:
    print(f"\nDoc needed for query #{id_query}:\n")
    doc_needed = client.beta.chat.completions.parse(
        
        model="gpt-4o",
        
        store=True,
        messages=[
            {
            "role": "system",
            "content": """
Sei un assistente esperto di cucina intergalattica che sta aiutando a capire quali documenti sono utili per rispondere ad una domanda di un utente relativi a dei piatti di cucina. Ci troviamo in un contesto fantascientifico, quindi i piatti sono inventati, come anche gli ingredienti che contengono e le tecniche di cucina che li compongono.
"""
            },
            {
            "role": "user",
            "content": f"""
Data la seguente domanda dell'utente:

### DOMANDA:

{new_questions[id_query]}

###

Oltre alla descrizione dei piatti, quali dei seguenti documenti a disposizione ritieni possano essere utili per rispondere alla domanda?

- 'manuale_sirius_cosmo: manuale di cucina di Sirius Cosmo: contiene descrizioni di tecniche di cucina (**considera che se non è nominato nella domanda di solito non è utile**).

- 'license_list': lista di tutte le possibili licenze che uno chef può possedere (ogni licenza è associata ad un livello).

- 'distanze': riporta le distanze in anni luce tra diversi pianeti (utile solo se la domanda contiene dettagli geografici tra pianeti).

- 'sostanze_quantita_regolamentate': documento sulla sostanze regolamentate che si possono usare in cucina nell'universo in cui ci troviamo e sulle loro **quantità** che è possibile utilizzare.

- 'licenze_necessarie_per_tecniche': lista di quali licenze è necessario possedere per utilizzare una specifica tecnica di cucina.

- 'sanzioni_pene': lista di sanzioni possibili.

- 'blog_etere_del_gusto': articolo che parla dei piatti serviti al ristorante "etere del gusto", contiene informazioni utili sui piatti serviti e sulle quantità volumetriche degli ingredienti che compongono i piatti.

- 'blog_sapore_del_dune': articolo che parla dei piatti serviti al ristorante "sapore del dune", contiene informazioni utili sui piatti serviti e sulle quantità volumetriche degli ingredienti che compongono i piatti.


quali dei documenti elencati ritieni possano essere utili per rispondere alla domanda?
Rispondi valorizzando gli attributi della classi di output con opportuni valori booleani.

"""
            }
        ],
        response_format=DocNeeded,
    )
    remaining_answers_doc_needed[id_query] = doc_needed
    print(doc_needed)

In [ ]:
class DocNeeded(BaseModel):
    manuale_sirius_cosmo: bool
    distanze_pianeti: bool
    sostanze_quantita_regolamentate: bool
    licenze_necessarie_per_tecniche: bool
    sanzioni_pene: bool
    blog_etere_del_gusto: bool
    blog_sapore_del_dune: bool
    license_list: bool

In [ ]:
%%time

class PlateOk(BaseModel):
    response: bool

remaining_responses = dict()

for id_query in question_remaining_ids:
    print(f"\nProcessing query: {id_query}")
    
    remaining_responses[id_query] = list()
    
    query = questions['domanda'][id_query-1]
    
    doc_needed = remaining_answers_doc_needed[id_query].choices[0].message.parsed
    
    auxiliary_texts = list()
    
    if doc_needed.manuale_sirius_cosmo:
        auxiliary_texts.extend(["### FRAMMENTI DAL MANUALE DI CUCINA DI SIRIUS COSMO\n\n"])
        auxiliary_texts.extend(search_similar_chunks(client, query, index_SC, texts_SC, k=10, model="text-embedding-ada-002"))
    
    if doc_needed.distanze_pianeti:
        auxiliary_texts.extend(["### DISTANZE TRA PIANETI:\n\n"])
        auxiliary_texts.extend(distanze)
        
    if doc_needed.sostanze_quantita_regolamentate:
        auxiliary_texts.extend(["### SOSTANZE E QUANTITA' REGOLAMENTATAE:\n\n"])
        auxiliary_texts.extend(sostanze_quantita_regolamentate)
        
    if doc_needed.licenze_necessarie_per_tecniche:
        auxiliary_texts.extend(["### FRAMMENTI DA LISTA DI LICENZE NECESSARIE PER USARE CERTE TECNICHE:\n\n"])
        auxiliary_texts.extend(search_similar_chunks(client, query, index_LNPT, texts_LNPT, k=10, model="text-embedding-ada-002"))
    
    if doc_needed.sanzioni_pene:
        auxiliary_texts.extend(["### LISTA DI SANZIONI POSSIBILI:\n\n"])
        auxiliary_texts.extend(sanzioni_pene)
        
    if doc_needed.blog_etere_del_gusto:
        auxiliary_texts.extend(["### BLOG RISTORANTE ETERE DEL GUSTO:\n\n"])
        auxiliary_texts.extend(blog_etere_del_gusto)
        
    if doc_needed.blog_sapore_del_dune:
        auxiliary_texts.extend(["### BLOG RISTORNATE SAPORE DEL DUNE:\n\n"])
        auxiliary_texts.extend(blog_sapore_del_dune)
        
    if doc_needed.license_list:
        auxiliary_texts.extend(["### LISTA DI TUTTE LE LICENZE CHE E' POSSIBILE AVERE:\n\n"])
        auxiliary_texts.extend(license_list)
    
    if len(auxiliary_texts) > 0:
        auxiliary_prompt = "### DOCUMENTI DI AIUTO:\n\n" + "\n---\n\n".join(auxiliary_texts)
        
    top_plates = retrieve_top_k_from_dict(query, embeddings_dict=descrizione_piatti_embeddings_dict, k=20)
    
    for plate in top_plates:
        output = client.beta.chat.completions.parse(
            
            model="gpt-4o", #"o3-mini",
            
            store=True,
            messages=[
                {
                    "role": "system",
                    "content": """
        Sei un assistente esperto di cucina che sta aiutando a capire se uno specifico piatto soddisfa o meno una richiesta di un cliente.
        I piatti di cui leggerai sono inventati e si trovano in una ambientazione fantascientifica nello spazio, in diversi sistemi solari.
        """
                },
                {
                    "role": "user",
                    "content": f"""
Date le seguenti informazioni, contenute in frammenti presi da documenti di vario tipo:

{auxiliary_prompt}

Data la seguente domanda:

### DOMANDA:
{query}

Il seguente piatto soddisfa la domanda dell'utente?

### DESCRIZIONE DEL PIATTO:

{plate_chunks_rag[plate]}

###

rispondi solo True o False.
"""
                }
            ],
            response_format=PlateOk,
        )
        if output.choices[0].message.parsed.response:
            remaining_responses[id_query].append(plate)
    print(remaining_responses[id_query])

In [ ]:
question_remaining_ids

In [ ]:
remaining_responses

In [ ]:
for id_query in question_remaining_ids:
    if len(remaining_responses[id_query]) > 0:
        answer = ','.join([str(dish_mapping.get(plate)) for plate in remaining_responses[i]])
        responses_pd.loc[responses_pd['row_id'] == id_query, 'result'] = answer

In [ ]:
responses_pd

In [ ]:
sum(responses_pd['result'] == '1,2,3')

In [ ]:
responses_pd.to_csv(f"/Users/vanzettom/Documents/mattia/personale/hackapizza/submissions/submission_questions_redefined_v5_rag.csv", index=False, sep=",")